# ADS-500B-02-FA21 {-}
# Final Data Science Programming Project {-}
# **Group 6: Claire Phibbs & Aaron Carr** {-}

# Dataset: Bank Marketing {-}

In [ ]:
%env

##### **Introduction**
For this final project, the bank_marketing.csv dataset was used to perform a secondary data analysis. This dataset provides information about marketing campaigns from a European bank. The dataset includes variables for age, job, marital status, education level, loan default status (i.e., yes/no), account balance, whether the loan is for housing (i.e., yes/no) or personal (i.e., yes/no), as well as information on the most recent bank outreach campaign, including contact method (i.e., cellular, telephone, unknown, Nan), last day of the month contaced, last month contacted, duration of last contact, number of contacts during current campaign, number of previous contacts (before the current campaign), outcome of the previous campaign, and whether the client has subscribed to a term deposit (i.e., yes/no). \
The goals of this project are to import and transform a raw dataset, perform exploratory and descriptive analysis, provide appropriate visualizations, and apply analytic models on the data. \
The main question being explored is whether one or more features--including demographics like age and education level, and previous campaign results--can be used to predict whether a bank client will take out a deposit.
A secondary question being explored is how factors like age, marital status, education level, account balance, and loan status affect defaulting on a loan from this European bank. \
Logistic regression models will be used to explore the relationship between the specified independent varibales and dependent variable for each question.

#### Import libariaries for data processing & analyses {-}

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from textwrap import wrap

from scipy.stats import chi2_contingency
from scipy.stats import mode

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr

In [ ]:
pandas2ri.activate()

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%R library(caTools)
%R library(ROCR)
%R library(car)
%R library(rms)
%R library("ggplot2")
%R library(tidyverse)

In [ ]:
%pwd

#### Set global variable values {-}

In [ ]:
round_int01 = 4
unk_str = 'unknown'

## 1. Data Importing & Pre-processing {-}

***
Null Hyphothesis (H0): There is no relationship between the X variables and the Y variable (i.e., deposit). \
Alternative Hyphothesis (Ha): There is a relationship between the X variables and the Y variable (i.e., deposit). \
***

### Initial data review {-}

In [ ]:
# Load dataframe (df) from csv file, print df
bank_df01 = pd.read_csv('bank_marketing.csv', header=0, sep=';')
print(bank_df01.head(10)) # display first 10 rows of df

bank_df01_cols_lst01 = bank_df01.columns.values.tolist() # review column names

bank_df01_len01 = len(bank_df01)
print(f'\n Number of df rows = {bank_df01_len01}') # display df length

In [ ]:
print(bank_df01.isnull().sum()) # review dataset for columns w/ null value

#### Initial Descriptive statistics {-}

In [ ]:
data_type = bank_df01.dtypes
print(data_type)

In [ ]:
bank_df01.describe()

##### **Data Import Explanation & Dataset Characteristics**
Initial steps included importing the bank_marketing.csv into Jupyter notebook as a pandas dataframe. The initial dataset has 45,211 rows and 17 columns (i.e., `age`, `job`, `marital`, `education`, `default`, `balance`, `housing`, `loan`, `contact`, `day`, `month`, `duration`, `campaign`, `pdays`, `previous`, `poutcome`, and `deposit`). \
There are a total of 4,028 missing values: `age` has 1,339 missing values; `default` has 1,306 missing values; and `contact` has 1,383 missing. The dtype command was used to see the different types of variables we are working with. \
The dataset contains a large number of categorical variables, including: `job`, `marital`, `education`, `default`, `housing`, `loan`, `contact`, `month`, `poutcome`, and `deposit`. The dataset also contains a few numeric variables: `age`, `balance`, `day`, `duration`, `campaign`, `pdays`, and `previous`. As `day` is ordinal, it will be treated as categorical varibale instead of discrete numerical. \
The describe command was also used to display the summary statistics of the numeric variables in the bank_df01 dataset. The summary statistics table shows the mean, standard deviation, min/max, and quartiles (reference table above). 

### Fill in missing data & transform ambiguous values {-}

### Create inital boxplots for features w/ numerical values {-}

In [ ]:
# Plot boxplots for numerical variables
fig, axs = plt.subplots(2, 3, figsize=(20 , 20), sharey=False, sharex=False) # set figure fram

axs[0, 0].boxplot(bank_df01['age'].dropna()) # subplot 1 for full dataset
axs[0, 0].set_title('Boxplt for age')

axs[0, 1].boxplot(bank_df01['balance'].dropna()) # subplot 2 for outlier dataset
axs[0, 1].set_title('Boxplot for balance')

axs[0, 2].boxplot(bank_df01['duration'].dropna()) # subplot 3 for outlier dataset
axs[0, 2].set_title('Boxplot for duration')

axs[1, 0].boxplot(bank_df01['campaign'].dropna()) # subplot 4 for outlier dataset
axs[1, 0].set_title('Boxplot for campaign')

axs[1, 1].boxplot(bank_df01['pdays'].dropna()) # subplot 5 for outlier dataset
axs[1, 1].set_title('Boxplot for pdays')

axs[1, 2].boxplot(bank_df01['previous'].dropna()) # subplot 6 for outlier dataset
axs[1, 2].set_title('Boxplot for previous')

plt.show()

##### **Initial visualizaton**
Check boxplots for numerical variables to indentify spread (varinace), and identify variables with outliers. \
Each one of the plots shows outliers for that variable. In order to mitigate their effects on the analyses, there will be removed. Focusing specifically on `previous` and `pdays`, the majority of values are zero based on the lack of variance, while there are also a signifacnt number of outliers. These variables will be removed, and instead a new feature will be created called `previous_contact` (yes=1/no=0) based on whether `pdays` equals -1 (client not previously contacted). \
Additional feature transformations are outlined below.

#### Define function: Fill in missing numerical data based on group by {-}

In [ ]:
def gb_agg_sub(df, t_var=None, gb_vars=[], agg_meth='mean'):
    '''current aggregate methods = sum, mean'''
    if agg_meth == 'sum':
        df_gpb01 = df.groupby(gb_vars).sum() # create a multi-indexed dataframe
    else:
        df_gpb01 = df.groupby(gb_vars).mean() # create a multi-indexed dataframe
    print(df_gpb01)
    df = pd.merge(df, df_gpb01, how='left', on=gb_vars, suffixes=(None, '_y'))
    df[t_var] = df[t_var].fillna(value=df[t_var + '_y'])
    return df

In [ ]:
# Run function to fill in missing age values based on group by
bank_df01 = gb_agg_sub(bank_df01, 'age',  ['marital', 'education'])

# Reset col names
bank_df01 = bank_df01[bank_df01_cols_lst01]

# Remove records where balance is less than zero
bank_df01 = bank_df01.loc[(bank_df01['balance'] >= 0), :]

# Create new feature
bank_df01['previous_contact'] = 0
bank_df01.loc[(bank_df01['pdays'] != -1), 'previous_contact'] = 1

# Save df len for generating % loss later
bank_df01_len02 = len(bank_df01)

# Remove col not used for analysis
bank_df01 = bank_df01.drop(['contact'], axis=1)
bank_df01 = bank_df01.drop(['pdays'], axis=1)
bank_df01 = bank_df01.drop(['previous'], axis=1)

# Fill in `default` w/ "unknown" to transform it below
bank_df01['default'] = bank_df01['default'].fillna(unk_str)

print(bank_df01.head())

#### Define function: Transform ambigous categorical values {-}

In [ ]:
def cat_mode(df, var=[(None, None, [])]):
    '''create function to transform variable values;
    var input uses "column string", "value to transform", "group_by vars" as x,y,z tuple'''
    df_sub01 = df.dropna()
    int_start01 = 1
    for i, j, k in var:
        df_sub02 = df_sub01.loc[(df_sub01[i] != j), :]
        df_gpb01 = df_sub02.groupby(k)[i].agg(lambda x: pd.Series.mode(x)[0]).to_frame() # create a multi-indexed dataframe; add lambda fx (Stack Overflow, n.d.)
        print(f'\ndf_gpb01:\n{df_gpb01}')
        df_sub01 = pd.merge(df_sub01, df_gpb01, how='left', on=k, suffixes=(None, '_z' + str(int_start01)))
        df_sub01.loc[(df_sub01[i] == j) & (df_sub01[i + '_z' + str(int_start01)].notna()), i] = df_sub01[i + '_z' + str(int_start01)]
        df_sub01 = df_sub01.drop(i + '_z' + str(int_start01), axis=1)
        int_start01 += 1
    return df_sub01

In [ ]:
bank_df01 = cat_mode(bank_df01, [('job', unk_str, ['marital', 'education']), ('education', unk_str, ['job', 'age']), ('default', unk_str, ['job', 'marital', 'education'])])

print(f'\nbank_df01:\n{bank_df01.head(10)}') # Review transformed df

In [ ]:
# Re-review dataset for columns w/ null value
print(bank_df01.isnull().sum())

##### **Explanation for filling in missing data**
In this dataset there are missing values for three of the variables (`age`, `default`, and `contact`). To handle the missing values for the `age` column we have used the groupby command to create a multi-indexed dataframe with the imputed `age` values. Basically, the marital status and education level columns are used to impute a value for `age`. The groupby command groups similar values together and takes a mean. The imputed values for `age` is then calculated by grouping similar attributes together and filling in the age column with the mean from the grouped `marital` and `education` columns. These imputed `age` values are in column `age_y` of the dataset. \
To handle the missing values for `contact`, we have decided to just remove the entire column. This decision was made due to the nature of our study questions. Since it is believed that the `contact` variable is not used to predict either default status or deposit decision, it is unnecssary to fill in the missing values or include them in the dataset at all. Instead they have been removed as to not introduce more bias into the dataset by imputing the values ourselves. \
The default column also has some missing values. A similar process was performed to fill in the missing data as with `age`, but for this variable the groupby was done using the following variables: `job`, `marital`, `education`. \
Also, something to note; we have decided to remove the rows in the balance column, where the balance is less than zero. 

## 2. Data Analysis & Visualizations {-}

### Create boxplots for remaining features w/ numerical values {-}

In [ ]:
# Plot boxplots for numerical variables
fig, axs = plt.subplots(2, 2, figsize=(20 , 20), sharey=False, sharex=False) # set figure fram

axs[0, 0].boxplot(bank_df01['age'].dropna()) # subplot 1 for full dataset
axs[0, 0].set_title('Boxplt for age')

axs[0, 1].boxplot(bank_df01['balance'].dropna()) # subplot 2 for outlier dataset
axs[0, 1].set_title('Boxplot for balance')

axs[1, 0].boxplot(bank_df01['duration'].dropna()) # subplot 3 for outlier dataset
axs[1, 0].set_title('Boxplot for duration')

axs[1, 1].boxplot(bank_df01['campaign'].dropna()) # subplot 4 for outlier dataset
axs[1, 1].set_title('Boxplot for campaign')

plt.show()

##### **Interpretation of Boxplots**
Above, boxplots have been created for four numerical variables in the dataset. This has been done, mainly to show the distribution of the data and to visualize outliers in the dataset. The boxplot for age shows that there is a relatively normal distribution, with outliers steming from the upper whisker. The boxplot for balance displays a highly skewed distribution with many outliers steming from the upper whisker. The balance variable also has a relatively small interquartile range compared to the other variables. The boxplot for duration displays a skewed distribution with many outliers steming from the upper whisker, and a mid-sized range compared to the other variables in the dataset. The boxplot for campaign also has a highly skewed distribution with many outliers steming from the upper whisker. Note that all of the boxplots are displayed ablove. 

#### Define function: Remove outliers {-}

In [ ]:
# Create function to generate comparison boxplots
def box_comp(df, var=[(None, 1.5)]):
    '''create function to id outliers & generate compartive boxplots;
    var input uses column string & outlier threshold as x,y tuple'''
    df_sub01 = df.dropna()
    df_sub01['outlier'] = 0 
    for i, j in var:
        q3, q1 = np.percentile(df_sub01[i], [75, 25]) # calculate quartiles 1 & 3
        iqr = q3 - q1 # calculate interquartile range
        print('\nIQR: {}-{} = {}'.format(round(q1, 4),round(q3, 4),round(iqr, 4))) # display IQR

        iqr_out = iqr * j # calculate outlier threshold
        otlr_low = q1 - iqr_out # calculate lower outlier limit
        otlr_high = q3 + iqr_out # calculate upper outlier limit
        df_sub01_sub1 = df_sub01.loc[(df_sub01[i] < otlr_low) | (df_sub01[i] > otlr_high)] # use .loc method to search for records that are outliers; assign to new dataframe
        df_sub01_sub2 = df_sub01.loc[(df_sub01[i] >= otlr_low) & (df_sub01[i] <= otlr_high)] # use .loc method to search for records that are outliers; assign to new dataframe
        df_sub01.loc[(df_sub01[i] < otlr_low) | (df_sub01[i] > otlr_high), 'outlier'] = 1
        len01 = len(df_sub01)
        len02 = len(df_sub01_sub1)
        len03 = len(df_sub01_sub2)

        fig2, axs = plt.subplots(1, 3, sharey=True, figsize=(12 , 10)) # set figure fram
        axs[0].boxplot(df_sub01[i].dropna()) # subplot 1 for full dataset
        axs[0].set_title('\n'.join(wrap(f'Boxplot for {i}: Full Dataset (N = {len01})', 30)))
        axs[1].boxplot(df_sub01_sub1[i].dropna()) # subplot 2 for outlier dataset
        axs[1].set_title('\n'.join(wrap(f'Boxplot for {i}: Outliers Subset (n = {len02})', 30)))
        axs[2].boxplot(df_sub01_sub2[i].dropna()) # subplot 2 for outlier dataset
        axs[2].set_title('\n'.join(wrap(f'Boxplot for {i}: w/o Outliers Subset (n = {len03})', 30)))
        plt.show()

        df_sub01.describe()
        print(df_sub01[i].describe()) # descriptive stats for CRIM varibale
        print('\n', df_sub01_sub1[i].describe()) # display descriptive stats of data subset
        print('\n', df_sub01_sub2[i].describe()) # display descriptive stats of data subset

        print('\nmean {} = {}'.format(i, round(df_sub01[i].mean(), 4))) # average age for full dataset
        print('median {} = {}'.format(i, round(df_sub01[i].median(), 4))) # median age for full dataset

        print('\noutliers mean {} = {}'.format(i, round(df_sub01_sub1[i].mean(), 4))) # average age for outliers
        print('outliers median {} = {}'.format(i, round(df_sub01_sub1[i].median(), 4))) # median age for outliers
        print('Sub1 Column count = {}'.format(len(df_sub01_sub1.columns))) # alternative way to print only number of columns
        print('Sub1 Row count = {}'.format(len(df_sub01_sub1))) # alternative way to print only number of rows

        print('\nw/o outliers mean {} = {}'.format(i, round(df_sub01_sub2[i].mean(), 4))) # average age for outliers
        print('w/o outliers median {} = {}'.format(i, round(df_sub01_sub2[i].median(), 4))) # median age for outliers
        print('Sub2 Column count = {}'.format(len(df_sub01_sub2.columns))) # alternative way to print only number of columns
        print('Sub2 Row count = {}'.format(len(df_sub01_sub2))) # alternative way to print only number of rows
    
    df_sub01_sub3 = df_sub01.loc[(df_sub01['outlier'] == 0), :]
    len04 = len(df_sub01_sub3)
    for k, l in var:
        fig6 = plt.figure(figsize=(12 , 10)) # set figure fram
        plt.boxplot(df_sub01_sub3[k].dropna()) # subplot 1 for full dataset
        plt.title('\n'.join(wrap(f'Boxplot for {k}: Total Subset (n = {len04})', 30)))
        plt.show()

    return df_sub01_sub3

In [ ]:
bank_df01 = box_comp(bank_df01, [('age', 1.5), ('balance', 1.5), ('campaign', 1.5)]) # revise main df to omit records with outliers for age variable
bank_df01 = bank_df01.drop('outlier', axis=1)

bank_df01_len03 = len(bank_df01)

##### **Removal  of Outliers**
Outliers were removed from `age`, `balance`, and `campaign` based on the function defined above that calculates the IQR for each variable, using a passed-in outlier threshold (e.g., IQR +/- 1 and a half times the IQR), and iterates to assign a flag of 0 or 1 to the total dataset based whether each record is within the outlier range for each applicable variable. This iteration (doing several separate assignments and only one elimination at the end) allows for minimizing how many records are removed from the final dataset, as there may be instances where one record may have outlier values in multiple features. Also, this method ensures that a distribution skew is not introduced through eliminating outliers for one variable, eliminating those records, then proceeding to the next variable--that method would mean that each time the mean may change for each subsequent variable in the (arbitrarily ordered) processing line. The net result is that instead of 7,463 records being eliminated (468 + 4,210 + 2,785), 7,085 were removed.

### Initial correlation matrix {-}

In [ ]:
# Generate initial correlation matrix
bank_df01_cols_lst02 = bank_df01.columns.values.tolist() # review column names
print(bank_df01_cols_lst02)
print(len(bank_df01_cols_lst02))
bank_df01.loc[:, bank_df01_cols_lst02].corr() # generate correlation matrix

##### **Interpretation of Correlation Matrix**
A correlation matrix is displayed above to show the independent relationship between the numerical variables in the dataset. From the correlation matrix, it can be observed that there is not a strong relationship between any them. The strongest observed relationship is between `campaign` and `day` (r = 0.1020), which is still considered to be very weak.

### Scatterplots of numerical variables {-}

#### Define function: Produce scatterplots {-}

In [ ]:
def scatr_vars(df, var_x, var_y):
    '''create function to print scatterplots comparing 2 vars'''
    y = df[var_y]
    x = df[var_x]
    x = sm.add_constant(x)
    lr_model = sm.OLS(y, x).fit()
    '''generate correlation coefficient & statsmodel regression table'''
    print('<<<<<-------------------------------------------------------------------->>>>>')
    print('r = {}'.format(round(df[var_x].corr(df[var_y]), 4))) # generate correlation coefficient
    '''create true regression line'''
    x_prime = np.linspace(x[var_x].min(), x[var_x].max(), 100) 
    x_prime = sm.add_constant(x_prime)
    y_hat = lr_model.predict(x_prime)
    '''plot x,y & regression line'''
    fig = plt.figure(figsize = (20 , 10))
    plt.scatter(df[var_x], df[var_y])
    plt.xlabel('x axis = {}'.format(var_x))
    plt.ylabel('y axis = {}'.format(var_y))
    plt.title('Scatterplot Between {} & {}'.format(var_x, var_y))
    plt.plot(x_prime[:,1], y_hat, 'red', alpha=.4)
    plt.show()

In [ ]:
# Run scatterplot fx
scatr_vars(bank_df01, 'age', 'balance')
scatr_vars(bank_df01, 'campaign', 'age')
scatr_vars(bank_df01, 'campaign', 'balance')

##### **Interpretation of scatterplots**
From reviewing the above scatterplot graphs and along with the outputed correlation coefficient, none of the "natural" numeric varibales (`age`, `balance`, `campaign`) seem to have a linear relationship with any other. The highest is a very weak positive correlation between `age` and `balance` (*r* = .0868). Note, these plots were done using the cleaned and transformed dataset (*n* = 34,360).

#### Define function: Discretize categorical feature values {-}

In [ ]:
# Define functions to convert categorical values to discrete numerical
col_arr_str = '_arr' # suffix to add to col names for array of cat variables

def pcat_list(df, col):
    '''define function to convert cat varibales to discrete numeric for correlation'''
    cat_var_col = col
    cat_var_uniq = df[cat_var_col].unique()
    cat_len = len(cat_var_uniq)
    int_start = 0
    cat_dict = {}
    cat_array = np.eye(cat_len, dtype=int)
    for i in cat_var_uniq:
        cat_dict[i] = cat_array[int_start]
        col_name01 = col + '_der'
        col_name02 = col + col_arr_str
        df.loc[(df[col].isin(cat_dict.keys())), col_name01] = df[col]
        int_start += 1
        df[col_name02] = df[col_name01].map(cat_dict)
    return col_name02


def pcat_df(df, col):
    '''define function to map cat varibales to discrete numeric'''
    cat_var_col = col
    cat_var_uniq = df[cat_var_col].unique()
    cat_len = len(cat_var_uniq)
    int_start = 0
    cat_dict = {}
    cat_array = np.eye(cat_len, dtype=int)
    for i in cat_var_uniq:
        cat_dict[i] = int_start
        col_name01 = col + '_der02'
        col_name02 = col + '_num_map'
        df.loc[(df[col].isin(cat_dict.keys())), col_name01] = df[col]
        int_start += 1
        df[col_name02] = df[col_name01].map(cat_dict)
    print(cat_dict)
    return df


def corr_vars(df, num_list=[], cat_list=[], intract={}):
    '''define function to output a correlation matrix for selected num & cat vars'''
    df_sub01 = df[num_list]
    cat_list_arr = []
    col_names01 = []
    col_names02 = []
    col_names03 = []
    merg_list_cont01 = []
    merg_list_cont02 = []
    merg_list_cont03 = []
    col_dict01 = {}
    last_col_list01 = []
    last_col_list02 = []
    '''loop to accumlate cat cols list'''
    for i in cat_list: # iterable in basic list of cat vars
        cat_list_arr.append(i + col_arr_str) # save col name + suffix to list
        col_names02 = list(df[i].unique()) # set list with unique values from cat vars cols
        col_names04 = [] # initalize list
        '''loop to add prefix to cat vals'''
        for m in col_names02: # iterabale in list of unique cat vals
            cat_full_name = str(i) + '_' + str(m)
            col_names04.append(cat_full_name)
        for b in intract:
            d_val = intract[b]
            for z in d_val:
                col_names05 = [] # initalize list
                if i == z:
                    for u in col_names04:
                        col_names05.append(str(b) + '_' + str(u) + '_in')
                    merg_list_cont01.extend(col_names04)
                else:
                    pass
                merg_list_cont02.extend(col_names05)
                last_col_list02.extend(col_names05[-1:]) # keep track of cols at the end after each pass
                merged_list = [(merg_list_cont02[i], merg_list_cont01[i]) for i in range(0, len(merg_list_cont02))]
                col_dict01[b] = merged_list
        col_names03.append(col_names04) # save list of col + cat vals to list
        col_names01.append(col_names02)
    int_start = 0
    '''loop to extend cat vars to new cols'''
    for i in cat_list_arr: # iterable in new cols list
        df02 = df[i].apply(pd.Series) # extend the vals to new cols
        num_list.extend(col_names03[int_start])
        df_sub01 = pd.concat([df_sub01, df02], axis=1) # splice new ext cols to orig cols
        df_sub01.columns = num_list 
        last_col_list01.extend(num_list[-1:]) # keep track of cols at the end after each pass
        int_start += 1
    '''add new cols for vars w/ interactions'''
    for s in col_dict01:
        d_val02 = col_dict01[s]
        for t, w in d_val02:
            df_sub01.loc[:, t] = df_sub01[s] * df_sub01[w]
    df_sub02 = df_sub01.drop(last_col_list01, axis=1) # drop based on c-1 cats
    df_sub02 = df_sub02.drop(last_col_list02, axis=1) # drop based on c-1 cats
    col_names = list(df_sub01.columns)
    print(f'\nColumn Names:\n{col_names}')

    return df_sub01.loc[:, col_names].corr(), df_sub01, df_sub02

### Key variable definitions {-}

#### Variables for predicting `deposit` values {-}

In [ ]:
# Load variable lists for functions & analysis
y_cor_vars_m2 = 'deposit'

num_vars_process_m2_01 = [y_cor_vars_m2, 'age', 'balance', 'campaign', 'previous_contact'] # dependent variable & core numerical variables

cat_vars_process_m2_01 = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'day', 'month'] # relevant caterigal varaibles for df02
cat_vars_process_m2_02 = [y_cor_vars_m2, 'job', 'marital', 'education', 'default', 'housing', 'loan', 'day', 'month'] # relevant caterigal varaibles for df06

In [ ]:
# Run scatterplot fx
scatr_vars(bank_df01, 'age', 'balance')

#### Subset Descriptive statistics {-}

In [ ]:
bank_df01.describe()

### Dataset branching {-}

In [ ]:
bank_df22 = bank_df01.copy().dropna()

bank_df22_len01 = len(bank_df22)

print(f'Original N = {bank_df01_len01}\nNew n = {bank_df22_len01}\n% Loss = {round((1-(bank_df22_len01/bank_df01_len01))*100, 2)}%')

##### **Final explanation of record cleaning & trimming**
The final percent loss of records was 24%, reducing the count from 45,211 to 34,360. Considering the large N that was started with, the fact that three variables had missing values, most of the numerical variables had large amounts of data points +/- 1.5 time the IQR, and the remaining count is still large, it has been decided that the percent loss is acceptable for the analyses to follow.

In [ ]:
# Dataframe for consolidate correlation of cat vars
for i in cat_vars_process_m2_02:
    bank_df26 = pcat_df(bank_df22, i)

bank_df28 = bank_df26.copy().dropna()

print(bank_df26.head(10))
print(bank_df28.head(10))

### Discretize key categorical features {-}

#### Run correlation & regression functions

In [ ]:
# Initialize new list
new_col_list02 = []

# Loop using function to populate new col list
for i in cat_vars_process_m2_01:
    nm02 = pcat_list(bank_df22, i)
    new_col_list02.append(nm02)

interact_dict02 = {'balance': ['job', 'education', 'marital'], 'age': ['job', 'education']}
full_corr01_m2, bank_df22, bank_df23  = corr_vars(bank_df22, num_vars_process_m2_01, cat_vars_process_m2_01)

### Updated correlation matrices {-}

In [ ]:
# Generate referential correlation matrix w/ full cat mapping (not c-1)
bank_df26_cols_lst = bank_df26.columns.values.tolist() # review column names
print(bank_df26_cols_lst)
print(len(bank_df26_cols_lst))

In [ ]:
full_corr01_m2

##### **Interpretation of Overall Correlation Matrix**
The Correlation matrix above was outputed to show the relationship between all of the variables in the dataset, after the imputation and alteration of the categorical variables and the missing values. Due to the discretized categorical variables, the correlation matrix loses its ability to inform on the relationship between two predictor variables. So, while the correlation table still provides a useful measure of the relationship between numerical variables (i.e. `age`, `balance`, `campaign`, ...) it not longer provides a useful measurement for the relationship betweent he discretized categorical variables.

## 3. Data Analytics {-}

#### **Data Legend:** \
bank_df01--> original bank_marketing.csv imported as a pandas df \
bank_df28--> discretized variables \
bank_df22--> full discretized dataset with all dummy variables \
bank_df23--> dummies; c-1

In [ ]:
bank_df42 = bank_df22.copy()
bank_df42 = pcat_df(bank_df42, 'deposit')

##### **R & Python integration**
To get the bank dataset compatible with R, the `deposit` variable outputs have been changed from categorical to binary, with deposit "no" as 0 and deposit "yes" as 1, using the pcat_df function. A new dataframe has been created called bank_df_r from the bank_df42 dataframe. Then, the bank_df_r dataset can be read into R using the RPY2 package.

### Develop R Logistic Regression Models {-}

In [ ]:
bank_df_r = bank_df42
print(bank_df_r.head(5))

In [ ]:
# Calculate ratio of yes/no in deposit var
yes_len = len(bank_df42.loc[(bank_df42['deposit'] == 'yes'), :])
no_len = len(bank_df42.loc[(bank_df42['deposit'] == 'no'), :])
full_len = len(bank_df42)
print(f'Subset len for "yes" = {yes_len}')
print(f'Subset len for "no" = {no_len}')
print(f'Subset len for full dataset = {full_len}')
print(f'yes % = {round((yes_len/full_len)*100, 4)}%')
print(f'no % = {round((no_len/full_len)*100, 4)}%')

In [ ]:
# Review value count of dependent var
print(len(bank_df42.loc[(bank_df42[y_cor_vars_m2] == 'yes'), :]))
print(len(bank_df42.loc[(bank_df42[y_cor_vars_m2] != 'yes'), :]))

# Subset default set
bank_df42c = bank_df42.loc[(bank_df42[y_cor_vars_m2] == 'yes'), :]
bank_df42d = bank_df42.loc[(bank_df42[y_cor_vars_m2] != 'yes'), :]
frac_target_42 = .4
n01_m1df42_sample = int((len(bank_df42c) / frac_target_42)) - len(bank_df42c)
bank_df42e = bank_df42d.sample(n = n01_m1df42_sample)
bank_df42f = pd.concat([bank_df42c, bank_df42e])
print(f'\nbank_df42f\nlen = {len(bank_df42f)}\n\ndf:\n{bank_df42f.head(5)}')

bank_df_r2 = bank_df42f
print(bank_df_r2.head(5))

In [ ]:
r_df = robjects.conversion.py2rpy(bank_df_r)
r_df2 = robjects.conversion.py2rpy(bank_df_r2)

In [ ]:
%R -i r_df
%R head(r_df)

In [ ]:
%R -i r_df2
%R (head(r_df2))

##### **Using RPY2 magic function**
After creating the bank_df_r dataframe, the robjects.conversion command was used to make the python bank_df_r dataframe into an R compatible dataframe, now called r_df. After the conversion, the dataframe can now be read into R using cell magic (i.e., %R--denoting R codeblock). The r_df dataframe has been printed in R to double check that the file has been read in correctly, with all the variables and outputs as a table, not an array.

In [ ]:
%R hist(r_df$age)
%R hist(r_df$balance)
%R hist(r_df$campaign)

##### **Interpretation of histograms for numeric variables (`age`, `balance`, `campaign`)**
The histogram for age displays a slight right skew, so the dataset has a slightly larger number of younger individuals than older individuals. The histogram for balance displays a strong right skew, so the dataset has a much larger number of lower balances than higher balances. The histogram for campaign displays a right skew as well, so most of the campaign values cluster to the left. This means that of the individuals in the dataset, more of them were contacted less frequently during the campaign for the client.

In [ ]:
%R names(r_df)[names(r_df)=="job_blue-collar"] <- "job_blue_collar"
%R names(r_df)[names(r_df)=="job_self-employed"] <- "job_self_employed"
%R head(r_df)

In [ ]:
%R sample1 <- floor(0.70 * nrow(r_df))
%R set.seed(1234)
%R train_ind <- sample(seq_len(nrow(r_df)), size=sample1)
%R train_r_df <- r_df[train_ind, ]
%R test_r_df <- r_df[-train_ind, ]

%R sample2 <- floor(0.70 * nrow(r_df2))
%R set.seed(1234)
%R train_ind2 <- sample(seq_len(nrow(r_df2)), size=sample2)
%R train_r_df2 <- r_df2[train_ind2, ]
%R test_r_df2 <- r_df2[-train_ind2, ]

In [ ]:
%R train_r_df

In [ ]:
%R test_r_df

In [ ]:
%R test_df_colum_nums <- colnames(test_r_df)
%R print(test_df_colum_nums)

##### **Creating Train & Test Subsets**
First, the column names for job_blue-collar and job_self-employed have been changed to job_blue_collar and job_self_employed, using the tidyverse package in R. This was done to create uniformity in the names of the columns in the dataset, important for modeling the data later. Second, training and testing subsets were created from the r_df dataframe, using the caTools package. The r_df data was split 70/30 for the train_r_df (24,052 rows) and test_r_df (10,308 rows). The last step taken before modeling the deposit variable using logistic regressions was to print the test_df_r column names and corresponding column number. This step was taken to have reference to the columns used in the logistic regression models below.

#### Model 1: Simple Logistic Regression (deposit ~ previous_contact) {-}

In [ ]:
%R log_r <- glm(deposit_num_map ~ previous_contact, data=train_r_df, family=binomial)
%R print(log_r)
%R log_r_summary <- summary(log_r)
%R print(log_r_summary)

#### Model 1: Receiver Operator Curve & Area Under the Curve {-}

In [ ]:
%R p <- predict(log_r, newdata=subset(test_r_df, select=c(5)), type="response")
%R pr <- prediction(p, test_r_df$deposit_num_map)
%R prf <- performance(pr, measure= "tpr", x.measure="fpr")
%R plot(prf)
%R auc <- performance(pr, measure="auc")
%R auc <- auc@y.values[[1]]

#### Model 1 Accuracy Measurement

In [ ]:
%R log_r_test <- predict(log_r, newdata=subset(test_r_df, select=c(5)), type="response")
%R log_r_test <- ifelse(log_r_test>0.5,1,0)
%R misclasificerror <- mean(log_r_test!=test_r_df$deposit_num_map)
%R print(paste('Accuracy', 1-misclasificerror))

##### **Model 1 Interpretation**
Model 1 is a simple logistic regression where the R application glm method is used to determine deposit status (i.e., "yes" or "no") as the dependent variable values from `previous_contact` as the independent variables in a training subset (train_r_df) of the full trimmed and cleanded dataset (*N* = 34,360). The intercept was calculated to be -2.264 and the regression coefficient for `previous_contact` to be 1.043, with a statistically significant *p*-value <.001. This means that as the number of previous contacts increases, there is a positive increase in the log odds (and by extension probability) of the client subscribing to a term deposit. Model 1 indicates that `previous_contact` is significant in determining the deposit status. The receiver operating curve (ROC) was then plotted to calculate the area under the curve (AUC). The ROC curve plots the true positive rate against the false positive rate, with AUC values closer to 1 indicating a good predictive model. The AUC for model 1 is equal to 0.6131, so the area under the ROC covers about 61% of the rectangle, indicating that model 1 does not have great predictive ability. The accuracy for the model is also reported to measure how well the model developed using the training data performed in predicting values within the test_r_df subset. The accuracy for model 1 is equal to 0.8786, which is high but does not necessarily indicate a well-fitted model due to the low value of the AUC. \
In summary, model 1's ability to predict deposit is not great, and this is most likely due to the fact that only predictor variable is included but there is more than one factor affecting if the client decides to subscribe to a term deposit.

#### Model 2: Simple Logistic Regression (deposit ~ campaign) {-}

In [ ]:
%R log_r_2 <- glm(deposit_num_map ~ campaign, data=train_r_df, family=binomial)
%R print(log_r_2)

%R log_r_2_summary <- summary(log_r_2)
%R print(log_r_2_summary)


#### Model 2: Receiver Operator Curve & Area Under the Curve

In [ ]:
%R log_r_2_test <- predict(log_r_2, newdata=subset(test_r_df, select=c(4)), type="response")
%R log_r_2_test <- ifelse(log_r_2_test>0.5,1,0)

%R p_log_r_2 <- predict(log_r_2, newdata=subset(test_r_df, select=c(4)), type="response")
%R pr_log_r_2 <- prediction(p_log_r_2, test_r_df$deposit_num_map)
%R prf_log_r_2 <- performance(pr_log_r_2, measure= "tpr", x.measure="fpr")
%R plot(prf_log_r_2)

%R auc_log_r_2 <- performance(pr_log_r_2, measure="auc")
%R auc_log_r_2 <- auc_log_r_2@y.values[[1]]

#### Model 2 Accuracy Measurement

In [ ]:
%R log_r_2_test <- predict(log_r_2, newdata=subset(test_r_df, select=c(4)), type="response")
%R log_r_2_test <- ifelse(log_r_2_test>0.5,1,0)
%R misclasificerror_2 <- mean(log_r_2_test!=test_r_df$deposit_num_map)
%R print(paste('Accuracy', 1-misclasificerror_2))

##### **Model 2 Interpretation**
Model 2 is a simple logistic regression where the R application glm method is used to determine deposit status (i.e., "yes" or "no") as the dependent variable values from `campaign` as the independent variable in a training subset (train_r_df) of the full trimmed and cleaned dataset (*N* = 34,360). The intercept was calculated to be -1.66857  and the regression coefficient for `campaign` to be -0.15859, with a statistically significant *p*-value <.001. This means that as the number of contacts performed during the campaign increases, there is a decrease in the log odds (and by extension probability) of the client subscribing to a term deposit. Model 2 indicates that `campaign` is significant in determining the deposit status. The ROC was then plotted to calculate the AUC. The AUC for model 2 is equal to 0.5645, so the area under the ROC covers about 56% of the rectangle, indicating that model 2 has a poor predictive ability.  The accuracy of model 2 is equal to 0.8786, which is high but does not necessarily indicate a well-fitted model due to the low value of the AUC. \
In summary, model 2's ability to predict deposit is poor, and this is most likely due to the fact that only one predictor variable is included in the regression equation, similar to model 1. This second simple logistic regression was performed to better understand which predictor variables have large affects on deposit status, by itself. This information will later be used in the multiple logistic regression models.

#### Model 3: Simple Logistic Regression (deposit ~ balance) {-}

In [ ]:
%R log_r_3 <- glm(deposit_num_map ~ balance , data=train_r_df, family=binomial)
%R print(log_r_3)

%R log_r_3_summary <- summary(log_r_3)
%R print(log_r_3_summary)

#### Model 3: Receiver Operator Curve & Area Under the Curve

In [ ]:
%R log_r_3_test <- predict(log_r_3, newdata=subset(test_r_df, select=c(3)), type="response")
%R log_r_3_test <- ifelse(log_r_3_test>0.5,1,0)

%R p_log_r_3 <- predict(log_r_3, newdata=subset(test_r_df, select=c(3)), type="response")
%R pr_log_r_3 <- prediction(p_log_r_3, test_r_df$deposit_num_map)
%R prf_log_r_3 <- performance(pr_log_r_3, measure= "tpr", x.measure="fpr")
%R plot(prf_log_r_3)

%R auc_log_r_3 <- performance(pr_log_r_3, measure="auc")
%R auc_log_r_3 <- auc_log_r_3@y.values[[1]]

#### Model 3 Accuracy Measurement

In [ ]:
%R log_r_3_test <- predict(log_r_3, newdata=subset(test_r_df, select=c(3)), type="response")
%R log_r_3_test <- ifelse(log_r_3_test>0.5,1,0)
%R misclasificerror_3 <- mean(log_r_3_test!=test_r_df$deposit_num_map)
%R print(paste('Accuracy', 1-misclasificerror_3))

##### **Model 3 Interpretation**
Model 3 is another simple logistic regression where the R application glm method is used to determine deposit status (i.e., "yes" or "no") as the dependent variable values from `balance` as the independent variable in a training subset (train_r_df) of the full trimmed and cleaned dataset (*N* = 34,360). The intercept was calculated to be -2.20501 and the regression coefficient for `balance` to be 0.00025, with statistically significant *p*-value <.001. This means that as the average yearly balance in Euros increases, there is a slight increase in the log odds (and by extension probability) of the client subscribing to a term deposit. Model 3 indicates that `balance` is significant in determining the deposit status. The ROC was then plotted to calculate the AUC. The AUC for model 3 is equal to 0.5701, so the area under the ROC covers about 57% of the rectangle, indicating that model 3 has a poor predictive ability. The accuracy of model 3 is equal to 0.8786, which is high but does not necessarily indicate a well-fitted model due to the low value of the AUC. \
In summary, model 3's ability to predict deposit is poor, and this is again most likely due to the fact that only one predictor variable is included in the regression equation, like model 1 and model 2. Now that simple logsitic regressions have been modeled to see which predictor variables have larger individual impacts on the deposit status, more complex multiple logistic regressions can be modeled, illustrating a more realistic relationship between deposit status and multiple predictor variables. 

#### Model 4: Multiple Logistic Regression (deposit ~ age+72 other variables....) {-}

In [ ]:
%R initial_logreg_model <- glm(deposit_num_map ~ age+balance+previous_contact+campaign+job_management+job_technician\
                               +job_entrepreneur+job_blue_collar+job_student+job_retired+job_admin.\
                               +job_services+job_self_employed\
                               +job_unemployed+job_housemaid+marital_married+marital_single\
                               +marital_divorced+education_tertiary+education_secondary\
                               +education_primary+education_unknown+default_yes+default_no\
                               +housing_yes+housing_no+loan_no+loan_yes+day_1+day_2+day_3\
                               +day_4+day_5+day_6+day_7+day_8+day_9+day_10+day_11\
                               +day_12+day_13+day_14+day_15+day_16+day_17+day_18\
                               +day_19+day_20+day_21+day_22+day_23+day_24+day_25\
                               +day_26+day_27+day_28+day_29+day_30+day_31+month_may\
                               +month_jun+month_jul+month_aug+month_oct\
                               +month_nov+month_dec+month_jan+month_feb\
                               +month_mar+month_apr+month_sep, data=train_r_df, family=binomial)
%R print(initial_logreg_model)

%R initial_logreg_model_summary <- summary(initial_logreg_model)
%R print(initial_logreg_model_summary)

%R ld.vars <- attributes(alias(initial_logreg_model)$Complete)$dimnames[[1]]
%R print(ld.vars)

#### Model 4: Receiver Operator Curve & Area Under the Curve

In [ ]:
%R p_initial_logreg_model <- predict(initial_logreg_model, newdata=subset(test_r_df, select=c(2,3,4,5,6,7,8,9,10,11\
                                                                                              ,12,13,14,15,16,17,18\
                                                                                              ,19,20,21,22,23,24,25\
                                                                                              ,26,27,28,29,30,31,32\
                                                                                              ,33,34,35,36,37,38,39\
                                                                                              ,40,41,42,43,44,45,46\
                                                                                              ,47,48,49,50,51,52,53\
                                                                                              ,54,55,56,57,58,59,60\
                                                                                              ,61,62,63,64,65,66,67\
                                                                                              ,68,69,70,71,72)), type="response")
%R pr_initial_logreg_model <- prediction (p_initial_logreg_model, test_r_df$deposit_num_map)
%R prf_initial_logreg_model <- performance(pr_initial_logreg_model, measure="tpr", x.measure="fpr")
%R plot(prf_initial_logreg_model)

%R auc_initial <- performance(pr_initial_logreg_model, measure="auc")
%R auc_initial <- auc_initial@y.values[[1]]

#### Model 4 Accuracy Measurement

In [ ]:
%R train_r_df <- r_df[train_ind, ]
%R test_r_df <- r_df[-train_ind, ]

%R initial_logreg_model_test_4 <- predict(initial_logreg_model, newdata=subset(test_r_df, select=c(2,3,4,5,6,7,8,9,10\
                                                                                                   ,11,12,13,14,15,16\
                                                                                                   ,17,18,19,20,21,22\
                                                                                                   ,23,24,25,26,27,28\
                                                                                                   ,29,30,31,32,33,34\
                                                                                                   ,35,36,37,38,39,40\
                                                                                                   ,41,42,43,44,45,46\
                                                                                                   ,47,48,49,50,51,52\
                                                                                                   ,53,54,55,56,57,58\
                                                                                                   ,59,60,61,62,63,64\
                                                                                                   ,65,66,67,68,69,70\
                                                                                                   ,71,72)), type="response")
%R initial_logreg_model_test_4 <- ifelse(initial_logreg_model_test_4>0.5,1,0)

%R misclasificerror_4 <- mean(initial_logreg_model_test_4!=test_r_df$deposit_num_map)
%R print(paste('Accuracy', 1-misclasificerror_4))

##### **Model 4 Interpretation**
Model 4 presents the first multiple logistic regression where the R application glm method is used to determine the deposit status (i.e., "yes" or "no") as the dependent variable values from all 73 predictors as the independent variables in a training subset (train_r_df) of the full trimmed and cleaned dataset (*N* = 34,360). The intercept was calculated to be -1.785e+00. The regression coefficients for the 73 independent variables can be found above along with their *p*-values and significance levels. Many of the independent variables have a significant *p*-value, indicating that a multiple logistic regression is more appropriate for this data than a simple logistic regression. The ROC was then plotted to calcuate the AUC. The AUC for model 4 is equal to 0.7458, so the area under the ROC covers about 75% of the rectangle, indicating that model 4 has an moderate predictive ability. The accuracy of model 4 is equal to 0.8815, which is high but does not necessarily indicate a well-fitted model due to the low value of the AUC. \
In summary, model 4's ability to predict deposit is moderate, most likely due to the fact that there are linearly dependent variables being used, affecting the overall affect on deposit status. The attributes function was used to output the linearly dependent variables (i.e., `job_housemaid`, `marital_divorced`, `education_unknown`, `default_no`, `housing_no`, `loan_yes`, `day_31`, and `month_sep`) illustrated by the "NA" outputs in the model 4 summary table. Given that these variables were found to be linearly dependent, they have been removed for the next model (i.e., model 5).

#### Model 5: Trimmed Multiple Logistic Regression (deposit ~ age+64 other variables.... {-}

In [ ]:
%R logreg_model <- glm(deposit_num_map ~ age+balance+previous_contact+campaign+job_management\
                       +job_technician+job_entrepreneur+job_blue_collar+job_student\
                       +job_retired+job_admin.+job_services+job_self_employed\
                       +job_unemployed+marital_married+marital_single+education_tertiary\
                       +education_secondary+education_primary+default_yes+housing_yes\
                       +loan_no+day_1+day_2+day_3+day_4+day_5+day_6+day_7+day_8+day_9\
                       +day_10+day_11+day_12+day_13+day_14+day_15+day_16+day_17+day_18\
                       +day_19+day_20+day_21+day_22+day_23+day_24+day_25+day_26+day_27\
                       +day_28+day_29+day_30+month_may+month_jun+month_jul+month_aug\
                       +month_oct+month_nov+month_dec+month_jan+month_feb+month_mar\
                       +month_apr, data=train_r_df, family=binomial)

%R logreg_model_summary <- summary(logreg_model)
%R print(logreg_model_summary)

%R vif(logreg_model)

#### Model 5: Receiver Operator Curve & Area Under the Curve

In [ ]:
%R p_logreg_model <- predict(logreg_model, newdata=subset(test_r_df, select=c(2,3,4,5,6,7,8,9\
                                                                              ,10,11,12,13,14\
                                                                              ,15,17,18,20,21\
                                                                              ,22,25,26,28,30\
                                                                              ,31,32,33,34,35\
                                                                              ,36,37,38,39,40\
                                                                              ,41,42,43,44,45\
                                                                              ,46,47,48,49,50\
                                                                              ,51,52,53,54,55\
                                                                              ,56,57,58,59,61\
                                                                              ,62,63,64,65,66\
                                                                              ,67,68,69,70,71)), type="response")
%R pr_logreg_model <- prediction (p_logreg_model, test_r_df$deposit_num_map)
%R prf_logreg_model <- performance(pr_logreg_model, measure="tpr", x.measure="fpr")
%R plot(prf_logreg_model)

%R auc_logreg <- performance(pr_logreg_model, measure="auc")
%R auc_logreg <- auc_logreg@y.values[[1]]

#### Model 5 Accuracy Measurement

In [ ]:
%R trim_logreg_model_test_5 <- predict(logreg_model, newdata=subset(test_r_df, select=c(2,3,4,5,6,7,8,9\
                                                                                        ,10,11,12,13,14\
                                                                                        ,15,17,18,20,21\
                                                                                        ,22,25,26,28,30\
                                                                                        ,31,32,33,34,35\
                                                                                        ,36,37,38,39,40\
                                                                                        ,41,42,43,44,45\
                                                                                        ,46,47,48,49,50\
                                                                                        ,51,52,53,54,55\
                                                                                        ,56,57,58,59,61\
                                                                                        ,62,63,64,65,66\
                                                                                        ,67,68,69,70,71))\
                                       , type="response")
%R trim_logreg_model_test_5 <- ifelse(trim_logreg_model_test_5>0.5,1,0)
%R misclasificerror_5 <- mean(trim_logreg_model_test_5!=test_r_df$deposit_num_map)
%R print(paste('Accuracy', 1-misclasificerror_5))

##### **Model 5 Interpretation**
Model 5 presents a trimmed multiple logistic regression where the R application glm method is used to determine the deposit status (i.e., "yes" or "no") as the dependent variable values from 65 predictors as the independent variables in a training subset (train_r_df) of the full trimmed and cleaned dataset (*N* = 34,360). The variables that exhibited perfect linear dependence have been removed for model 5. The intercept was calculated to be -1.785e+00. The regression coefficients for the 65 independent variables can be found above along with their *p*-values and significance levels. Many of the independent variables have a significant *p*-value, indicating that a multiple logistic regression is still appropriate. \
The VIF's for the independent variables were also calculated to show the linear dependency of each variable in model 5. The ROC was then plotted to calcuate the AUC. The AUC for model 5 is equal to 0.7458, so the area under the ROC covers about 75% of the rectangle, indicating that model 5 has a moderate predictive ability. The accuracy of model 5 is equal to 0.8815, which is high but does not necessarily indicate a well-fitted model due to the low value of the AUC. \
In summary, model 5's ability to predict deposit is moderate, most likely due to the fact that there are still variables being used that exhibit some degree of linear dependence, affecting the overall affect on deposit status. To mitigate this issue in model 5, model 6 removes the variables with a calculated VIF>2.5. 

#### Model 6: Trimmed Multiple Logistic Regression 
(default~age+balance+campaign+previous_contact+job_entrepreneuer+default_yes+housing_yes+loan_no+month_dec+month_mar)

In [ ]:
#excluing variables with a vif>2.5#
%R trim_logreg_model <- glm(deposit_num_map ~ age + balance + campaign + previous_contact\
                            + job_entrepreneur + default_yes + housing_yes + loan_no\
                            + month_dec + month_mar, data=train_r_df, family=binomial())
%R print(trim_logreg_model)

%R trim_logreg_model_summary <- summary(trim_logreg_model)
%R print(trim_logreg_model_summary)

%R vif(trim_logreg_model)

#### Model 6: Receiver  operator Curve & Area Under the Curve

In [ ]:
%R p_trim_logreg_model <- predict(trim_logreg_model, newdata=subset(test_r_df, select=c(2,3,4,5,8,25,26\
                                                                                        ,28,67,70)), type="response")
%R pr_trim_logreg_model <- prediction (p_trim_logreg_model, test_r_df$deposit_num_map)
%R prf_trim_logreg_model <- performance(pr_trim_logreg_model, measure="tpr", x.measure="fpr")
%R plot(prf_trim_logreg_model)

%R auc_trim_6 <- performance(pr_trim_logreg_model, measure="auc")
%R auc_trim_6 <- auc_trim_6@y.values[[1]]

#### Model 6 Accuracy Measurement

In [ ]:
%R trim_logreg_model_test_6 <- predict(trim_logreg_model, newdata=subset(test_r_df, select=c(2,3,4,5,8,25,26,28\
                                                                                             ,67,70)), type="response")
%R trim_logreg_model_test_6 <- ifelse(trim_logreg_model_test_6>0.5,1,0)
%R misclasificerror_6 <- mean(trim_logreg_model_test_6!=test_r_df$deposit_num_map)
%R print(paste('Accuracy', 1-misclasificerror_6))

##### **Model 6 Interpretation**
Model 6 presents another trimmed multiple logistic regression where the R application glm method is used to determine the deposit status (i.e., "yes" or "no") as the dependent variable values from `age`, `balance`, `campaign`, `previous_contact`, `job_entrepreneur`, `default_yes`, `housing_yes`, `loan_no`, `month_dec`, and `month_mar` as the independent variables in a training subset (train_r_df) of the full trimmed and cleaned dataset (*N* = 34,360). The variables that exhibited a VIF>2.5 have been removed for model 6. The intercept was calculated to be -1.810e+00. \
The regression coefficient for `age` is -0.00957 with a statistically significant *p*-value <.001. This means that as age increases, there is a slight decrease in the log odds (and by extension probability) of the client subscribing to a term deposit. Model 6 indicates that `age` is significant in determining the deposit status. \
The regression coefficient for `balance` is 0.00019 with a statistically significant *p*-value <.001. This means that as average yearly balance in Euros increases, there is a slight increase in the log odds of the client subscribing to a term deposit. Model 6 indicates that `balance` is significant in determining the deposit status. \
The regression coefficient for `campaign` is -0.12680 with a statistically significant *p*-value <.001. This means that as campaign increases, there is a decrease in the log odds of the client subscribing to a term deposit. Model 6 indicates that `campaign` is significant in determining the deposit status. \
The regression coefficient for `previous_contact` is 1.0620 with a statistically significant *p*-value <.001. This means that based on whether there was previous contact there is a moderate increase in the log odds of the client subscribing to a term deposit. Model 6 indicates that `previous_contact` is significant in determining the deposit status. \
The regression coefficient for `job_entrepreneur` is -0.29270 with a statistically significant *p*-value <.05. This means that when a person's job is entrepreneur, there is a slight decrease in the log odds of the client subscribing to a term deposit. Model 6 indicates that `job_entrepreneur` is significant in determining the deposit status. \
The regression coefficient for `default_yes` is -0.41920 with a *p*-value of 0.1356. This means that if you default, there is a increase in the log odds of the client subscribing to a term deposit. Model 6 indicates that `default_yes` is not significant in determining the deposit status. \
The regression coefficient for `housing_yes` is -0.99900 with a statistically significant *p*-value <.001. This means that people with a housing loan, decrease the log odds of the client subscribing to a term deposit. Model 6 indicates that `housing_yes` is significant in determining the deposit status. \
The regression coefficient for `loan_no` is 0.53270 with a statistically significant *p*-value <.001. This means that people without a personal loan, increase the log odds of the client subscribing to a term deposit. Model 6 indicates that `loan_no` is significant in determining the deposit status. \
The regression coefficient for `month_dec` is 0.92550 with a statistically significant *p*-value <.001. This means that people who were last contacted about the deposit in December, increase the log odds of the client subscribing to a term deposit. Model 6 indicates that `month_dec` is significant in determining the deposit status. \
The regression coefficient for `month_mar` is 1.5940 with a statistically significant *p*-value <.001. This means that people who were last contacted about the deposit in March, increase the log odds of the client subscribing to a term deposit. Model 6 indicates that `month_mar` is significant in determining the deposit status. \
The ROC was then plotted to calcuate the AUC. The AUC for model 6 is equal to 0.7145, so the area under the ROC covers about 71% of the rectangle, indicating that model 6 has a moderate predictive ability. The accuracy of model 6 is equal to 0.8805, which is high but does not necessarily indicate a well-fitted model due to the low value of the AUC. \
In summary, model 6's ability to predict deposit is moderate, most likely due to including the `default_yes` variable. Since this variable was not significant, it will be removed for model 7, to more accurately depict a logistic model for deposit status. 

#### **Model 7: Trimmed Multiple Regression 
(deposit~age+balance+camapgin+previous-contact+job_entrepreneuer+housing_yes+loan_no+month_dec+month_mar)

In [ ]:
# Final model!!
%R trim_logreg_model_7 <- glm(deposit_num_map ~ age +balance + campaign + previous_contact\
                              + job_entrepreneur + housing_yes + loan_no + month_dec\
                              + month_mar, data=train_r_df, family=binomial())
%R print(trim_logreg_model_7)

%R trim_logreg_model_7_summary <- summary(trim_logreg_model_7)
%R print(trim_logreg_model_7_summary)

%R vif(trim_logreg_model_7)

#### Model 7: Receiver Operator Curve & Area Under the Curve

In [ ]:
%R p_trim_logreg_model_7 <- predict(trim_logreg_model_7, newdata=subset(test_r_df, select=c(2,3,4,5,8,26,28,67\
                                                                                            ,70)), type="response")
%R pr_trim_logreg_model_7 <- prediction (p_trim_logreg_model_7, test_r_df$deposit_num_map)
%R prf_trim_logreg_model_7 <- performance(pr_trim_logreg_model_7, measure="tpr", x.measure="fpr")
%R plot(prf_trim_logreg_model_7)

%R auc_trim_7 <- performance(pr_trim_logreg_model_7, measure="auc")
%R auc_trim_7 <- auc_trim_7@y.values[[1]]

#### Model 7 Accuracy Measurement

In [ ]:
%R trim_logreg_model_test_7 <- predict(trim_logreg_model_7, newdata=subset(test_r_df, select=c(2,3,4,5,8,26\
                                                                                               ,28,67,70))\
                                       , type="response")
%R trim_logreg_model_test_7 <- ifelse(trim_logreg_model_test_7>0.5,1,0)
%R misclasificerror_7 <- mean(trim_logreg_model_test_7!=test_r_df$deposit_num_map)
%R print(paste('Accuracy', 1-misclasificerror_7))

##### **Model 7 Interpretation**
Model 7 presents the "best" trimmed multiple logistic regression where the R application glm method is used to determine the deposit status (i.e., "yes" or "no") as the dependent variable values from  `age`, `balance`, `campaign`, `previous_contact`, `job_entrepreneur`, `housing_yes`, `loan_no`, `month_dec`, and `month_mar` as the independent variables in a training subset (train_r_df) of the full trimmed and cleaned dataset (*N* = 34,360). The `default_yes` variable has been removed for model 7. The intercept was calculated to be -1.8210. \
The regression coefficient for `age` is -0.00954 with a statistically significant *p*-value <.001. This means that as age increases, there is a slight decrease in the log odds (and by extension probability) of the client subscribing to a term deposit. Model 7 indicates that `age` is significant in determining the deposit status. \
The regression coefficient for `balance` is 0.00019 with a statistically significant *p*-value <.001. This means that as average yearly balance in Euros increases, there is a slight increase in the log odds (and by extension probability) of the client subscribing to a term deposit. Model 7 indicates that `balance` is significant in determining the deposit status. \
The regression coefficient for `campaign` is -0.1267 with a statistically significant *p*-value <.001. This means that as campaign increases, there is a decrease in the log odds of the client subscribing to a term deposit. Model 7 indicates that `campaign` is significant in determining the deposit status. \
The regression coefficient for `previous_contact` is 1.06442 with a statistically significant *p*-value <.001. This means that based on whether there was previous there is a moderate increase in the log odds of the client subscribing to a term deposit. Model 7 indicates that `previous_contact` is significant in determining the deposit status. \
The regression coefficient for `job_entrepreneur` is -0.29379 with a statistically significant *p*-value <.05. This means that when a person's job is entrepreneur, there is a slight decrease in the log odds of the client subscribing to a term deposit. Model 7 indicates that `job_entrepreneur` is significant in determining the deposit status. \
The regression coefficient for `housing_yes` is -0.99772 with a statistically significant *p*-value <.001. This means that people with a housing loan, decrease the log odds of the client subscribing to a term deposit. Model 6 indicates that `housing_yes` is significant in determining the deposit status. \
The regression coefficient for `loan_no` is 0.53737 with a statistically significant *p*-value <.001. This means that people without a personal loan, increase the log odds of the client subscribing to a term deposit. Model 7 indicates that `loan_no` is significant in determining the deposit status. \
The regression coefficient for `month_dec` is 0.92773 with a statistically significant *p*-value <.001. This means that people who were last contacted about the deposit in December, increase the log odds of the client subscribing to a term deposit. Model 7 indicates that `month_dec` is significant in determining the deposit status. \
The regression coefficient for `month_mar` is 1.59689 with a statistically significant *p*-value <.001. This means that people who were last contacted about the deposit in march, increase the log odds of the client subscribing to a term deposit. Model 7 indicates that `month_mar` is significant in determining the deposit status. \
The ROC was then plotted to calcuate the AUC. The AUC for model 7 is equal to 0.7150, so the area under the ROC covers about 72% of the rectangle, indicating that model 7 has a moderate predictive ability. The accuracy of model 7 is equal to 0.8805, which is high but does not necessarily indicate a well-fitted model due to the low value of the AUC. \
In summary, model 7 is the best of the 8 models produced in its ability to predict deposit status. So, when `age`, `balance`, `camapaign`, `previous_contact`, `job_entrepreneuer`, `housing_yes`, `loan_no`, `month_dec`, and `month_mar` are used to predict deposit status, it produces the best of the models, since all the predictors have a significant effect on deposit status. Something to note: While this model produces the best results compared to our other models, model 7 still doesn't have great predictive ability of deposit status overall. 

#### **Model 7b: Trimmed Multiple Regression 
(deposit ~ age+balance+campaign+previous-contact+job_entrepreneuer+housing_yes+loan_no+month_dec+month_mar)

In [ ]:
#final model on subset where "yes" ~40% of sample!!#
%R trim_logreg_model_7b <- glm(deposit_num_map ~ age +balance + campaign + previous_contact\
                               + job_entrepreneur + housing_yes + loan_no + month_dec\
                               + month_mar, data=train_r_df2, family=binomial())
%R print(trim_logreg_model_7b)

%R trim_logreg_model_7b_summary <- summary(trim_logreg_model_7b)
%R print(trim_logreg_model_7b_summary)

%R vif(trim_logreg_model_7b)

#### Model 7b: Receiver Operator Curve & Area Under the Curve

In [ ]:
%R p_trim_logreg_model_7b <- predict(trim_logreg_model_7b, newdata=subset(test_r_df2, select=c(2,3,4,5,8,26,28\
                                                                                               ,67,70))\
                                     , type="response")
%R pr_trim_logreg_model_7b <- prediction (p_trim_logreg_model_7b, test_r_df2$deposit_num_map)
%R prf_trim_logreg_model_7b <- performance(pr_trim_logreg_model_7b, measure="tpr", x.measure="fpr")
%R plot(prf_trim_logreg_model_7b)

%R auc_trim_7b <- performance(pr_trim_logreg_model_7b, measure="auc")
%R auc_trim_7b <- auc_trim_7b@y.values[[1]]

#### Model 7b Accuracy Measurement

In [ ]:
%R trim_logreg_model_test_7b <- predict(trim_logreg_model_7b, newdata=subset(test_r_df2, select=c(2,3,4,5,8\
                                                                                                  ,26,28,67,70))\
                                        , type="response")
%R trim_logreg_model_test_7b <- ifelse(trim_logreg_model_test_7b>0.5,1,0)
%R misclasificerror_7b <- mean(trim_logreg_model_test_7b!=test_r_df2$deposit_num_map)
%R print(paste('Accuracy', 1-misclasificerror_7b))

#### Model 8: Trimmed Multiple Logistic Regression (deposit ~ balance+campaign+previous_contact)

In [ ]:
%R trim_logreg_model_8 <- glm(deposit_num_map ~ balance + campaign + previous_contact, data=train_r_df\
                              , family=binomial())
%R print(trim_logreg_model_8)

%R trim_logreg_model_8_summary <- summary(trim_logreg_model_8)
%R print(trim_logreg_model_8_summary)

%R vif(trim_logreg_model_8)

#### Model 8: Receiver Operator Curve & Area Under the Curve

In [ ]:
%R p_trim_logreg_model_8 <- predict(trim_logreg_model_8, newdata=subset(test_r_df, select=c(3,4,5)), type="response")
%R pr_trim_logreg_model_8 <- prediction (p_trim_logreg_model_8, test_r_df$deposit_num_map)
%R prf_trim_logreg_model_8 <- performance(pr_trim_logreg_model_8, measure="tpr", x.measure="fpr")
%R plot(prf_trim_logreg_model_8)

%R auc_trim_8 <- performance(pr_trim_logreg_model_8, measure="auc")
%R auc_trim_8 <- auc_trim_8@y.values[[1]]

#### Model 8 Accuracy Measurement

In [ ]:
%R trim_logreg_model_test_8 <- predict(trim_logreg_model_8, newdata=subset(test_r_df, select=c(3,4,5)), type="response")
%R trim_logreg_model_test_8 <- ifelse(trim_logreg_model_test_8>0.5,1,0)
%R misclasificerror_8 <- mean(trim_logreg_model_test_8!=test_r_df$deposit_num_map)
%R print(paste('Accuracy', 1-misclasificerror_8))

##### **Model 8 Interpretation**
Model 8 presents a trimmed multiple logistic regression where the R application glm method is used to determine the deposit status (i.e., "yes" or "no") as the dependent variable values from `balance`, `campaign`, and `previous_contact` as the independent variables in a training subset (train_r_df) of the full trimmed and cleaned dataset (*N* = 34,360). Model 8 looks at the relationship between three of the numerical predictors and deposit status. The intercept was calculated to be -2.1950. \
The regression coefficient for `balance` is 0.00023 with a statistically significant *p*-value <.001. This means that as average yearly balance in Euros increases, there is a slight increase in the log odds (and by extension probability) of the client subscribing to a term deposit. Model 8 indicates that `balance` is significant in determining the deposit status. \
The regression coefficient for `campaign` is -0.12150 with a statistically significant *p*-value <.001. This means that as campaign increases, there is a decrease in the log odds of the client subscribing to a term deposit. Model 8 indicates that `campaign` is significant in determining the deposit status. \
The regression coefficient for `previous_contact` is 0.09879 with a statistically significant *p*-value <.001. This means that based on whether there was previous contact there is a slight increase in the log odds of the client subscribing to a term deposit. Model 8 indicates that `previous_contact` is significant in determining the deposit status. \
The ROC was then plotted to calcuate the AUC. The AUC for model 8 is equal to 0.6626, so the area under the ROC covers about 66% of the rectangle, indicating that model 8 has a weak predictive ability. The accuracy of model 8 is equal to 0.8786, which is high but does not necessarily indicate a well-fitted model due to the low value of the AUC. \
In summary, model 8's ability to predict deposit is weak, most likely due to the fact that there are excluded variables that have an affect on deposit status. This model was created to show how soley the numeric variables affect deposit status.

##### **Overall Results & Summary**
Several models were developed to reduce the number of features to a manageable amount and attempt to find the model with the highest chance of accurately predicting the binary outcome of the `deposit` variable (dependent feature). Models 1-3 were simple logistic regression models comparing one response variable to one predictor variable. As outlined under each model above, none of them were sufficiently predictive to be contenders for the final and most useful model. Models 4-8 used multiple independent variables with the idea that their combinations would be better at predicting the binary result than any one of them alone. Model 4 was the largest model and included 73 independent variables. The large number was due to a large number of categorical variables that were discretized by generating dummy variables, and moreover, several of the nominal variables had several individual categories (e.g., after transforming "unknown" values for job and education, they had 11 and 4 categories respectively). Model 4 was run to see if all variables included would generate a good predictive model, but after doing so, it only had an accuracy of 88.15%, which was only slightly higher than the overall ratio of "yes" in the full cleaned and trimmed dataset (87.96%; *N* = 34,360). For model 5, the number of predictor variables was reduced to 65 by eliminating those variables (relative to model 4) that were determined to be perfectly linearly dependent on other independent variables (namely, `job_housemaid`, `marital_divorced`, `education_unknown`, `default_no`, `housing_no`, `loan_yes`, `day_31`, `month_sep`), as indicated by a regression coefficient of "NA". Module 6 was reduced relative to model 5 down to 10 independent variables by eliminating any variables with a variance inflation factor (VIF) greater than 2.5. This was done to further avoid possible multi-collinearity with other independent variables. Model 6 had a *p*-value of .1356 for `default_yes`, so it was eliminated for model 7 and 7b--as these latter models were determined to be the best from our analyses, there are discussed further below. For one last model (model 8), solely the "naturally" numerical variables were used to see if they alone would provide a sufficient model. It was found that the accuracy was 87.86% and AUC was 0.6626 which did not make for a good model.

##### **Final Models**
Model 7 used the following predictor variables along with the corresponding regression coefficients: `age` (-0.0095), `balance` (0.0002), `campaign` (-0.1267), `previous_contact` (1.0644), `job_entrepreneur` (-0.2938), `housing_yes` (-0.9977), `loan_no` (0.5374), `month_dec` (0.9277), and `month_mar` (1.5969). Intercept = -1.8210. After reviewing all of the models, it was determined that model 7 had the best combination of accuracy (88.05%), AUC (0.7150), and interpretability. Though the AUC was closer to 0.5 than 1.0, it was somewhat close to the models with the best overall AUC (models 4 & 5 AUC = 0.7458). Obviously this does not make it good, per se, but given the predictability of all models, it was mid- to high-level relatively. \
In terms of interpreting the model, as age increased there was a slight reduction in the probability of being able to predict deposit as “yes” and with an increase in balance, probability rose a little; both coefficients are slight. An increase in the number of contacts during the campaign actually decreased the probability, which makes sense if considering that people might be frustrated at being contacted more and would then be less likely to do business with that bank. However, whether the person did have any previous contact (“yes” or “no”) did seem to have relatively high positive impact. Also interesting was that whether or not the person’s job was entrepreneur had a negative affect—it should be noted that since only one job type was included, the only interpretation for this model that can be made is based on whether someone was an entrepreneur or not (i.e., a binary distinction, such that all other job types were lumped together into a “not-entrepreneur” category). This could be because as the job of entrepreneur there may be large fluctuations in income due to whether their current ventures are going well or not. Whether someone had a housing loan or didn’t have a personal loan also had an impact on the probability of predicting deposit as “yes”. This makes sense from the angle that a housing loan indicates a substantial outlay of income (thereby increasing probability that additional funds are not available to make a deposit), but a personal loan may be the opposite in effect. Lastly, whether the contact month was December, March, or other has had an impact. One possibility is that those were high volume banking months based on holiday bonuses received (in December) as well as preparation and saving for summer vacations (in March), and that contact in those months would remind people to open deposit accounts close to when they actually needed a specific place to keep funds. \
Model 7b was an attempt to mitigate the fact that the “yes” value in the full trimmed and cleaned dataset represented only 12.04% of the sample. As one additional step, the records with “yes” were pulled out, the n was divided by a fractional signifier (.4 in this case) to achieve a total desired n such that the ratio of “yes” to “no” was 40%. The set that contained only “no” values was then randomly sampled to pull out the number of records such that they would represent 60% of the total sample (1 - .4). Then using that dataset (*n* = 10,345) the same process that model 7 went through was performed: namely, it was split into a train and test subsets, and then a model was created using the same independent variables, as follows with regression coefficients: `age` (-0.0148), `balance` (0.0002), `campaign` (-0.1324), `previous_contact` (1.034*), `job_entrepreneur` (-0.3484), `housing_yes` (-0.9395), `loan_no` (0.5553), `month_dec` (1.296*), and `month_mar` (1.874*). Intercept = -0.0847. * Rounding due to RPY2 output in scientific notation. The intercept decreased significantly, and the two month-related coefficients increased relatively significantly. However, the accuracy and AUC actually decreased (68.14% and 69.39% respectively) relative to model 7, so in the end, model 7b was worse at predicting the probability of the dependent variable compared to model 7.

##### **Strengths & possible limitations of the current analyses**
There were several strengths to the dataset. The size of the dataset (*N* = 45,211) meant that there would be sufficient data to develop a regression model using both a training set and a testing set. Additionally, both Python and R were used, which provided the ability to utilize each program for its strengths; Python's numpy and pandas libarries were used for the pre-processing due their speed and efficient, and R was used to generate the logistic regression models and generate several of the corresponding visualizations. With the trimming models used, multicollinearity was sufficiently addressed to avoid interaction or interdependence between predictor variables. \
There were also several factors that may have contributed a bias to the results. Initially a heavy cleaning of the data was required, which removed 20.86% of the data and also required filling in missing values (e.g., for age), and transforming ambigous values (e.g., changing "unknown" value in job to other values based on the modes of grouped data); taking a slightly different approach my have resulted in different outcomes. Another limitation may have been the available ratio of categorical values in some of the columns--for instance, if the number of yes's and no's in binary variable column were not even, it may have meant that the model again did not have sufficient compartive sample for each value to develop a robust predictive model. Lastly, there were a lot of variables to choose from initially to build the model--most of which may be a predictor of the dependent variables--but also several of them were categorical variables (some of which had several categories) that had to be discretized to be used in the model building process, all of which meant that including all of the dummy variables there were more than 70 to choose from to develop the model.

##### **Future directions**
Explore addition classification methods to obtain a better predictive model that was achieved using logistic regression.

***
**References:** \
* Coban, H. (2019). Here's how I used Python to build a regression model using an e-commerce dataset. https://searchengineland.com/heres-how-i-used-python-to-build-a-regression-model-using-an-e-commerce-dataset-326493 \
* Devore, J. L. (2016). *Probability and statistics*. (9th ed.). Cengage Learning. \
* Mukala, M. M. (2012). A guide to appropriate use of correlation coefficient in medical research. Malawi Medical Journal, 24(3), 69-71. \
* Real Python. (n.d.). Linear regression in python. https://realpython.com/linear-regression-in-python/ \
* Real Python. (n.d.). Logistic regression in python. https://realpython.com/logistic-regression-python/ \
* Shah, C. (2020). *A hands-on introduction to data science*. Cambridge University Press. \
* Stack Overflow. (n.d.). GroupBy pandas dataFrame and select most common value. https://stackoverflow.com/questions/15222754/groupby-pandas-dataframe-and-select-most-common-value \
* UCLA: Statistical Consulting Group. Introduction to SAS. https://stats.idre.ucla.edu/sas/modules/sas-learning-moduleintroduction-to-the-features-of-sas/ (accessed December 12, 2021).\
***